In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# 1. 데이터 불러오기
df = pd.read_csv("시간대별 범죄+생활인구밀도+조도&소음 데이터_2022.csv")


In [ ]:
# 2. 총 범죄 건수 변수 생성
# 강력범죄에 해당하는 열만 선택
crime_cols = [
    '살인기수', '살인미수등', '강도', '강간', '유사강간', '강제추행',
    '기타 강간 강제추행등', '방화', '상해', '폭행', '체포 감금', '협박',
    '약취 유인', '폭력행위등', '공갈'
]

# 범죄 총합 계산
df['총범죄건수'] = df[crime_cols].sum(axis=1)


In [ ]:
# 3. 사용할 생활인구밀도 항목 정의 (5개)
pop_cols = [
    '내국_유소년_생활인구밀도',
    '내국_생산가능_생활인구밀도',
    '내국_고령_생활인구밀도',
    '외국_TEMP_생활인구밀도',
    '외국_LONG_생활인구밀도'
]

In [ ]:
### A. 인구군별(min-max) 정규화
scaler_col = MinMaxScaler()
df_pop_by_col = pd.DataFrame(scaler_col.fit_transform(df[pop_cols]), columns=["인구군" + col for col in pop_cols])


In [ ]:
### B. 시간대별(row 기준) 정규화
df_pop_by_row = df[pop_cols].copy()
df_pop_by_row = df_pop_by_row.sub(df_pop_by_row.min(axis=1), axis=0)
df_pop_by_row = df_pop_by_row.div(df_pop_by_row.max(axis=1), axis=0)
df_pop_by_row.columns = ["시간대_" + col for col in pop_cols]


In [ ]:
# 정규화 대상 인구군만 추출
pop_subset = df[pop_cols].copy()

# 행 단위로 min-max 정규화 수행 (시간대별 비교용)
pop_subset_norm = pop_subset.sub(pop_subset.min(axis=1), axis=0)
pop_subset_norm = pop_subset_norm.div(pop_subset.max(axis=1), axis=0)

# 정규화된 컬럼명 지정
pop_subset_norm.columns = ["시간대_" + col for col in pop_subset.columns]

# 원본 데이터프레임에 정규화 결과 병합
df_time_normed = pd.concat([df, pop_subset_norm], axis=1)

# 정규화 결과 미리보기
df_time_normed[[col for col in pop_subset_norm.columns]].head()

,시간대_내국_유소년_생활인구밀도,시간대_내국_생산가능_생활인구밀도,시간대_내국_고령_생활인구밀도,시간대_외국_TEMP_생활인구밀도,시간대_외국_LONG_생활인구밀도
0,0.037308,0.813700,0.037709,0.0,0.373689
1,0.042237,0.824506,0.043865,0.0,0.356618
2,0.046169,0.830175,0.046064,0.0,0.361296
3,0.040366,0.819363,0.039823,0.0,0.374105


In [ ]:
### C. 기존 인구밀도 데이터 제거
df = df.drop(columns=pop_cols)

### D. 두 정규화 결과 병합
df = pd.concat([df, df_pop_by_col, df_pop_by_row], axis=1)

In [ ]:
# 4. 조도 및 소음 정규화 (원래 이름 유지)
env_cols = ['조도 평균(lux)', '소음 평균(dB)']
scaler_env = MinMaxScaler()
df[env_cols] = scaler_env.fit_transform(df[env_cols])

In [ ]:
print(df.columns.tolist())

['시간대', '내국_총생활인구밀도', '살인기수', '살인미수등', '강도', '강간', '유사강간', '강제추행', '기타 강간 강제추행등', '방화', '상해', '폭행', '체포 감금', '협박', '약취 유인', '폭력행위등', '공갈', '조도 평균(lux)', '소음 평균(dB)', '총범죄건수', '인구군내국_유소년_생활인구밀도', '인구군내국_생산가능_생활인구밀도', '인구군내국_고령_생활인구밀도', '인구군외국_TEMP_생활인구밀도', '인구군외국_LONG_생활인구밀도', '시간대_내국_유소년_생활인구밀도', '시간대_내국_생산가능_생활인구밀도', '시간대_내국_고령_생활인구밀도', '시간대_외국_TEMP_생활인구밀도', '시간대_외국_LONG_생활인구밀도']


In [ ]:
# 1. 이동할 컬럼 리스트
env_cols = ['조도 평균(lux)', '소음 평균(dB)']

# 2. 나머지 컬럼 리스트 (환경 컬럼 제외)
other_cols = [col for col in df.columns if col not in env_cols]

# 3. 새로운 컬럼 순서로 재정렬
df = df[other_cols + env_cols]

# 4. 확인
df.head()

,시간대,내국_총생활인구밀도,살인기수,살인미수등,강도,강간,유사강간,강제추행,기타 강간 강제추행등,방화,...,인구군내국_고령_생활인구밀도,인구군외국_TEMP_생활인구밀도,인구군외국_LONG_생활인구밀도,시간대_내국_유소년_생활인구밀도,시간대_내국_생산가능_생활인구밀도,시간대_내국_고령_생활인구밀도,시간대_외국_TEMP_생활인구밀도,시간대_외국_LONG_생활인구밀도,조도 평균(lux),소음 평균(dB)
0,00시-06시,239613.3297,36,83,236,1515,212,3776,83,295,...,0.000000,1.000000,0.598175,0.045850,1.0,0.046343,0.0,0.459247,0.000000,0.000000
1,06시-12시,245689.9019,48,73,103,710,119,2456,38,229,...,0.657643,0.282922,0.000000,0.051227,1.0,0.053202,0.0,0.432523,0.388069,0.709473
2,12시-18시,250198.4118,71,106,140,636,115,3006,57,282,...,1.000000,0.000000,0.637837,0.055614,1.0,0.055487,0.0,0.435204,1.000000,1.000000
3,18시-24시,243852.1900,92,148,188,1144,155,5090,70,417,...,0.336061,0.760403,1.000000,0.049266,1.0,0.048603,0.0,0.456581,0.051943,0.840692


In [ ]:
# 1. 옮기고 싶은 컬럼들
target_col = '내국_총생활인구밀도'
before_col = '조도 평균(lux)'

# 2. 현재 컬럼 리스트
cols = df.columns.tolist()

# 3. 위치 조정: 타겟 컬럼 제거 후, before_col 위치 앞에 삽입
cols.remove(target_col)
insert_idx = cols.index(before_col)
cols.insert(insert_idx, target_col)

# 4. 새로운 순서로 재정렬
df = df[cols]

# 5. 확인
df.head()

,시간대,살인기수,살인미수등,강도,강간,유사강간,강제추행,기타 강간 강제추행등,방화,상해,...,인구군외국_TEMP_생활인구밀도,인구군외국_LONG_생활인구밀도,시간대_내국_유소년_생활인구밀도,시간대_내국_생산가능_생활인구밀도,시간대_내국_고령_생활인구밀도,시간대_외국_TEMP_생활인구밀도,시간대_외국_LONG_생활인구밀도,내국_총생활인구밀도,조도 평균(lux),소음 평균(dB)
0,00시-06시,36,83,236,1515,212,3776,83,295,7574,...,1.000000,0.598175,0.045850,1.0,0.046343,0.0,0.459247,239613.3297,0.000000,0.000000
1,06시-12시,48,73,103,710,119,2456,38,229,5214,...,0.282922,0.000000,0.051227,1.0,0.053202,0.0,0.432523,245689.9019,0.388069,0.709473
2,12시-18시,71,106,140,636,115,3006,57,282,6709,...,0.000000,0.637837,0.055614,1.0,0.055487,0.0,0.435204,250198.4118,1.000000,1.000000
3,18시-24시,92,148,188,1144,155,5090,70,417,12018,...,0.760403,1.000000,0.049266,1.0,0.048603,0.0,0.456581,243852.1900,0.051943,0.840692


In [ ]:
# '_생활인구밀도' 문자열 제거
df.columns = df.columns.str.replace('_생활인구밀도', '', regex=False)

# 결과 확인
print(df.columns.tolist())

['시간대', '살인기수', '살인미수등', '강도', '강간', '유사강간', '강제추행', '기타 강간 강제추행등', '방화', '상해', '폭행', '체포 감금', '협박', '약취 유인', '폭력행위등', '공갈', '총범죄건수', '인구군내국_유소년', '인구군내국_생산가능', '인구군내국_고령', '인구군외국_TEMP', '인구군외국_LONG', '시간대_내국_유소년', '시간대_내국_생산가능', '시간대_내국_고령', '시간대_외국_TEMP', '시간대_외국_LONG', '내국_총생활인구밀도', '조도 평균(lux)', '소음 평균(dB)']


✅ 정규화 방식 요약 (메모용)
1. 인구군별 정규화 (_인구군정규화)
정의: 각 인구군(예: 내국_유소년, 외국_TEMP 등)에 대해 전체 시간대(행) 기준으로 Min-Max 정규화 수행

  범위: 0~1

  목적: 시간 흐름에 따라 각 인구군의 변화량을 파악

  → 예: 유소년 인구가 어떤 시간대에 가장 많고 적은지를 확인

2. 시간대별 정규화 (_시간대정규화)
정의: 각 시간대(행) 내에서 5개 인구군 간 값을 비교하여 Min-Max 정규화 수행

  범위: 0~1

  목적: 동일 시간대 내 인구군 간 상대적 분포 비율 파악

  → 예: 00~06시에 외국 TEMP가 가장 많은 인구군인지 확인


3. 조도/소음 정규화
정의: 조도 평균(lux)과 소음 평균(dB)을 전체 시간대 기준으로 Min-Max 정규화

  범위: 0~1

  목적: 조도/소음 값을 다른 변수와 비교하기 쉽게 표준화

